In [ ]:
! pip install langchain==0.1.2 openai

In [1]:
# 加载环境
from dotenv import load_dotenv

load_dotenv()

True

## PromptTemplate

In [2]:
from langchain.prompts import PromptTemplate

promt_template = PromptTemplate.from_template(
    'Tell me a {adjective} joke about {content}'
)

promt_template.format(adjective='funny', content='chickens')

'Tell me a funny joke about chickens'

In [3]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

## ChatPromptTemplate

In [5]:
from langchain.prompts import  ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name='Bob', user_input='What is your name?')
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'), HumanMessage(content='Hello, how are you doing?'), AIMessage(content="I'm doing well, thanks!"), HumanMessage(content='What is your name?')]


In [6]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=("You are a helpful assistant that re-writes the user's text to sound more upbeat.")
        ),
        HumanMessagePromptTemplate.from_template('{text}')
    ]
)

messages = chat_template.format_messages(text="I don't like eating tasty things.")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things.")]


## LCEL

In [7]:
prompt_val = promt_template.invoke({'adjective':'funnny', 'content':'chickens'})
prompt_val

StringPromptValue(text='Tell me a funnny joke about chickens')

In [8]:
prompt_val.to_string()

'Tell me a funnny joke about chickens'

In [9]:
prompt_val.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'base', 'StringPromptValue'],
 'kwargs': {'text': 'Tell me a funnny joke about chickens'}}

In [10]:
prompt_val.to_messages()

[HumanMessage(content='Tell me a funnny joke about chickens')]

In [12]:
chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})
chat_val

ChatPromptValue(messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')])

In [13]:
chat_val.to_messages()

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
 HumanMessage(content='i dont like eating tasty things.')]

In [14]:
chat_val.to_string()

"System: You are a helpful assistant that re-writes the user's text to sound more upbeat.\nHuman: i dont like eating tasty things."